# Implementing the "Cube" approach

In [1]:
import pykep as pk
import numpy as np
import json
import pickle as pkl

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot as plt
%matplotlib notebook

import cascade as csc
from copy import deepcopy
from tqdm.notebook import tqdm
import heyoka as hy

# added for the Cube approach implementation
from collections import defaultdict
import time
import sgp4
from sgp4.api import Satrec, SatrecArray

In [2]:
with open("data/debris_simulation_ic.pk", "rb") as file:
    r_ic,v_ic,collision_radius,to_satcat,satcat,debris = pkl.load(file)

In [3]:
def period(r,v, mu):
    """Computes the orbital period from the vis-viva equation

    Args:
        r (float): The radius (in L).
        v (float): The velocity (in L/T).
        mu (float): The gravitational parameter in L^3/T^2

    Returns:
        The orbital period (in T)
    """
    En = v**2/2 - mu / r
    a = -mu / En / 2
    if a<0:
        raise ValueError("Hyperbola!!!")
    return np.sqrt(a**3/mu)*2*np.pi

In [4]:
def cubes(cartesian_points, cube_dimension):
    """Runs the Cube algorithm and returns satellites within the same cube

    Args:
        cartesian_points (Nx3 np.array): The cartesian position of the satellites (in L).
        cube_dimension (float): The cube dimentsion (in L).

    Returns:
        a list containing lists of satelites idx occupying the same cube
    """
    # init
    retval = []
    cubes = defaultdict(list)

    # We compute the floored Cartesian coordinates identifying the bins.
    pos = cartesian_points
    pos = pos / cube_dimension
    pos = np.floor(pos).astype(int)
    # We fill the bins
    for i, xyz in enumerate(pos):
        cubes[tuple(xyz)].append(i)
    # We find bins with more than one atellite
    for key in cubes:
        if len(cubes[key]) > 1:
            retval.append(cubes[key])
    return retval

In [5]:
def simulate_sgp4(debris, sim_time=20,time_grid=5, t0 = 8073.603992389981):
    """Computes all satellites ephemerides on a time grid

    Args:
        debris (list of pk.planets): The objects to propagate.
        sim_time (float): The total propagation time (in years).
        time_grid(float): The time resolution (in days).
        t0 (float): the starting epoch in mjd2000.

    Returns:
        a list containing lists of idx identifying the object occupying the same cube
    """
    # This list will contain all the sgp4 Satrec objects
    satellite_l = []
    for deb in debris:
        l1 = deb.line1
        l2 = deb.line2
        satellite_l.append(Satrec.twoline2rv(l1, l2))
    # Here we build the vectorized version allowing for speed
    satellites = SatrecArray(satellite_l)
    jd0, fr = pk.epoch(t0).jd, 0.0
    # The Julian dates are from jd0 to 20 years after
    jds = jd0 + np.arange(0,sim_time*365.25/time_grid)*time_grid
    frs = jds * 0
    return satellites.sgp4(jds, frs)

In [6]:
%timeit cubes(r_ic / 1000, 10)

29 ms ± 4.37 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
start = time.time()
e,r,v = simulate_sgp4(np.array(debris)[to_satcat], sim_time=20,time_grid=5, t0 = 8073.603992389981+np.random.random()*5)
end = time.time()
print("Time elasped: ", end - start)


Time elasped:  8.106714963912964


In [8]:
# We assume all satellites are valid at the starting epoch
start = time.time()
undecayed = set(np.arange(r.shape[0]))
n_collisions=0
for i in range(r.shape[1]):
    Lcube = 10. #km
    time_grid  = 5 #days
    # If signalled from the sgp4, we remove the indices of the decayed satellites
    decayed = set(np.where(e[:,i]>0)[0])
    undecayed = undecayed - decayed
    undecayed_l = np.array([j for j in undecayed])
    # We detect all satellites couples in the same cube of Lcube km size
    collision = cubes(r[undecayed_l,i,:], cube_dimension = Lcube)
    #kdt = KDTree(r[undecayed_l,i,:])
    #collision = list(kdt.query_pairs(Lcube))
    #print(collision)
    for pair in collision:
        # we get the indexes in r,v
        idx1 = undecayed_l[pair[0]]
        idx2 = undecayed_l[pair[1]]
        # we store positions and velocities from r,v
        r1 = r[idx1,i,:]
        r2 = r[idx2,i,:]
        v1 = v[idx1,i,:]
        v2 = v[idx2,i,:]
        # we get the collision radiu from debris (indexed differently hence to_satcat is used)
        collision_radius1 = debris[to_satcat[idx1]].collision_radius
        collision_radius2 = debris[to_satcat[idx2]].collision_radius
        # Relative velocity 
        Vrel = np.linalg.norm(v1-v2)
        # Collisional area of the couple (in km^2)
        sigma = np.pi*((collision_radius1+collision_radius2)/1000)**2 
        # Volume of the cube (km^3)
        U = (Lcube)**3
        # We compute the spatial densities
        # densities (from "Assessing collision algorithms for the newspace era" )
        s1 = 1./U
        s2 = 1./U
        # collision probability
        Pij = s1*s2*Vrel*sigma*U*time_grid*pk.DAY2SEC
        # Store
        if Pij > np.random.random():
            print(f"Collision! pair: {pair}, years: {i*5/365.25}")
            n_collisions+=1
end = time.time()
print("Time elasped: ", end - start)
print("Decayed objects: ", r_ic.shape[0] - len(undecayed))
print("Number of collisions: ", n_collisions)

Collision! pair: [15735, 16592], years: 0.16427104722792607
Collision! pair: [13836, 14952], years: 0.8487337440109514
Collision! pair: [11689, 15638], years: 1.8069815195071868
Collision! pair: [1865, 14948], years: 2.3134839151266258
Collision! pair: [13494, 15408], years: 2.7104722792607805
Collision! pair: [7187, 15358], years: 2.833675564681725
Collision! pair: [14191, 15863], years: 2.847364818617385
Collision! pair: [3573, 17077], years: 2.915811088295688
Collision! pair: [13872, 15462], years: 4.955509924709103
Collision! pair: [12081, 16336], years: 5.900068446269678
Collision! pair: [12501, 13565], years: 7.693360711841205
Collision! pair: [12528, 14067], years: 10.403832991101986
Collision! pair: [10057, 14805], years: 13.114305270362765
Collision! pair: [13680, 13701], years: 13.374401095140314
Collision! pair: [1595, 13447], years: 15.824777549623546
Collision! pair: [715, 11435], years: 18.05612594113621
Time elasped:  54.44314360618591
Decayed objects:  4987
Number of co

In [9]:
r_ic.shape[0] - len(undecayed)

4987

In [10]:
r_ic.shape

(19669, 3)

In [11]:
q = mp.Queue()

NameError: name 'mp' is not defined

In [ ]:
q.put(2.)

In [ ]:
q.put(23.9)

In [ ]:
list(q)

TypeError: 'Queue' object is not iterable

In [ ]:
from scipy.spatial import KDTree

In [ ]:
kd = KDTree(r_ic/1000)

In [ ]:
list(kd.query_pairs(10))

[(16909, 16931),
 (5092, 6507),
 (7538, 9580),
 (18380, 18837),
 (17397, 17399),
 (18670, 18672),
 (18474, 18478),
 (18635, 18636),
 (4854, 6471),
 (15565, 15577)]

In [ ]:
cubes(r_ic/1000, 10)

[[4854, 6471], [15565, 15577]]

In [ ]:
4/3*np.pi/8

0.5235987755982988